# Setup

In [1]:
from tqdm import tqdm
import numpy as np
import pandas as pd
import time
import datetime
import sys
import matplotlib.pyplot as plt
import seaborn as sns
import os
import shutil
import json
import pickle


np.random.seed(42)

from formats import experiment_pb2
from formats import  quantification_pb2

from skimage import io
import pandas as pd
import utils


from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

import torch
from torch.utils.data import Dataset, DataLoader, ConcatDataset, ChainDataset


import os
import pyro
import pyro.distributions as dist
import pyro.poutine
from pyro.infer import MCMC, NUTS
import math
import torch.nn as nn
import torch.nn.functional as F
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam
from pyro.infer.autoguide.guides import AutoDiagonalNormal
import pyro.distributions.constraints as constraints
from tqdm import trange

import utils



In [2]:
local_radius_px = 5

In [3]:
import data
dataset = data.get_dataset(local_radius_px=local_radius_px)

In [114]:
x,q = next(iter(torch.utils.data.DataLoader(dataset=dataset,batch_size=len(dataset))))

# from sklearn.preprocessing import StandardScaler
# x_shape = x.shape
# x = StandardScaler().fit_transform(x.reshape(-1,x.shape[-1])).reshape(x_shape)
# q = StandardScaler().fit_transform(q)

# x = torch.Tensor(x)
# q = torch.Tensor(q)

x /= x.shape[1]*x.shape[2]/q.shape[1]


In [121]:
52/16

3.25

In [115]:
def eval_reconstruction(reconstruction):
    x_hat, q_hat = reconstruction
    
    x_flat = x.reshape(-1,x.shape[-1])
    x_hat = x_hat.reshape(-1,x.shape[-1])

    x_mean = x_flat.mean(0)
    x_ssr = (x_flat - x_hat).pow(2).sum()
    x_sst = (x_flat - x_mean).pow(2).sum()
    x_r2 = 1 - x_ssr/x_sst

    q_mean = q.mean(0)
    q_ssr = (q - q_hat).pow(2).sum()
    q_sst = (q - q_mean).pow(2).sum()
    q_r2 = 1 - q_ssr/q_sst

    return x_r2.item(), q_r2.item()


In [116]:
def joint_pca_reconstruct(n_components):
    combined_data = torch.cat((q,x.reshape((x.shape[0],-1))),1)
    pca = PCA(n_components=n_components)
    pca.fit(combined_data)
    recon = torch.tensor(pca.inverse_transform(pca.transform(combined_data)))

    q_hat = recon[:,:q.shape[1]] 
    x_hat = recon[:,q.shape[1]:].reshape(x.shape) 
    
    return x_hat, q_hat

def joint_pca_latents(n_components):
    combined_data = torch.cat((q,x.reshape((x.shape[0],-1))),1)
    pca = PCA(n_components=n_components)
    return pca.fit_transform(combined_data)

In [117]:
eval_reconstruction(
    joint_pca_reconstruct(n_components=1)
)

(0.7374055943106592, 0.02206323675223343)

In [118]:
eval_reconstruction(
    joint_pca_reconstruct(n_components=2)
)

(0.7524027768391566, 0.4358681221488049)

In [119]:
eval_reconstruction(
    joint_pca_reconstruct(n_components=3)
)

(0.7528608619827024, 0.6393485737060141)

In [113]:
for norm_factor in np.arange(1,100,5):
    x /= norm_factor

    def eval_reconstruction(reconstruction):
        x_hat, q_hat = reconstruction
        
        x_flat = x.reshape(-1,x.shape[-1])
        x_hat = x_hat.reshape(-1,x.shape[-1])

        x_mean = x_flat.mean(0)
        x_ssr = (x_flat - x_hat).pow(2).sum()
        x_sst = (x_flat - x_mean).pow(2).sum()
        x_r2 = 1 - x_ssr/x_sst

        q_mean = q.mean(0)
        q_ssr = (q - q_hat).pow(2).sum()
        q_sst = (q - q_mean).pow(2).sum()
        q_r2 = 1 - q_ssr/q_sst

        return x_r2.item(), q_r2.item()

    def joint_pca_reconstruct(n_components):
        combined_data = torch.cat((q,x.reshape((x.shape[0],-1))),1)
        pca = PCA(n_components=n_components)
        pca.fit(combined_data)
        recon = torch.tensor(pca.inverse_transform(pca.transform(combined_data)))

        q_hat = recon[:,:q.shape[1]] 
        x_hat = recon[:,q.shape[1]:].reshape(x.shape) 
        
        return x_hat, q_hat

    def joint_pca_latents(n_components):
        combined_data = torch.cat((q,x.reshape((x.shape[0],-1))),1)
        pca = PCA(n_components=n_components)
        return pca.fit_transform(combined_data)
    
    print(norm_factor)
    print(eval_reconstruction(joint_pca_reconstruct(n_components=1)))
    print(eval_reconstruction(joint_pca_reconstruct(n_components=2)))
    print(eval_reconstruction(joint_pca_reconstruct(n_components=3)))
    print()
    x *= norm_factor




1
(0.7521909570466426, 0.007489184130836213)
(0.8437800360955272, 0.021052586212242264)
(0.8685366549539695, 0.02139881718012482)

6
(0.7521883019714131, 0.007591373151785041)
(0.8436717229891644, 0.02488976069652027)
(0.8679818350784987, 0.03532927028618282)

11
(0.7521578249474512, 0.007863951067025754)
(0.8369275961995328, 0.07463169287954674)
(0.8432360017169178, 0.4515279940855007)

16
(0.7520161709634988, 0.008390531842191296)
(0.7571769271880807, 0.42675407091754725)
(0.8351439874557265, 0.47973636196687064)

21
(0.7515017974469874, 0.009419862670241197)
(0.754019660806886, 0.43406425820223127)
(0.7601663659860601, 0.6292715548553929)

26
(0.7494782243654843, 0.011924167623012183)
(0.7531694514303244, 0.43519132738531474)
(0.7538540881642922, 0.6384588602622177)

31
(0.7357511333563822, 0.023296996180023566)
(0.7523588116682863, 0.43590088237697233)
(0.7528147108263997, 0.6393829649136819)

36
(0.31912898525382294, 0.26980568723984155)
(0.751100248455126, 0.43669248480436396)
(0